In [1]:
import MDAnalysis as mda

# Assigning PDB Chain-IDs based on Topology within TPR file

A Gromacs TPR files contains the topology information of a system,including the chains.

MDAnalysis can read and interpret numerous file types, including PDB, GRO, TPR, TRR and XTC.

### Prepartions

* first generate LDH tetramer with Jmol:
```
load =3LDH filter "Biomolecule 1";
select protein;
write "3ldh_raw.pdb"
```
* next run through pdb2gmx, genbox and grompp:
```bash
# download minim.mdp from Justin Lemkul Lysozyme tutorial:
wget http://www.mdtutorials.com/gmx/lysozyme/Files/minim.mdp
gmx pdb2gmx -f 3ldh_raw.pdb -o LDH_oplsaa.pdb  -ff oplsaa -water tip3p
gmx editconf -f LDH_oplsaa.pdb -o LDH_oplsaa_box.pdb -c -d 1.0 -bt cubic
gmx grompp -f minim.mdp -c LDH_oplsaa_box.pdb -p topol.top -o ldh.tpr
```


### Procedure

* Read topology from TPR file `ldh.tpr`  
  (protein consisting of 4 chains, OPLS-AA forcefield, 
  with box information, used a minim.mdp)
* Read coordinates for "frame 0" from `LDH_oplsaa_box.pdb` 
  (the PDB from which the TPR was generated; could use a `.gro` as well)
* generate a list `chain_ids` of possible Chain-IDs (A-Z)
* Loop over "segments" (chains) in the Universe
  * `segindex` is a 0-based index of the current segment
  * `segid` is a (string) segment-ID
  * use segindex to select a single-letter chain-ID from the `chain_ids` list
    and assign it to `segid`
* write PDB file `ldh_fixed_chainIDs.pdb` with fixed chain-IDs.

In [2]:
%ls *.tpr

ldh.tpr


In [3]:
# Read Universe from tpr (topology) and PDB (coordinates)
u = mda.Universe('ldh.tpr', 'LDH_oplsaa_box.pdb')

In [4]:
# explore the Universe object
print(u)
print(u.residues)
print(u.segments)

<Universe with 20616 atoms>
<ResidueGroup [<Residue THR, 0>, <Residue ALA, 1>, <Residue LEU, 2>, ..., <Residue LEU, 1313>, <Residue LYS, 1314>, <Residue PHE, 1315>]>
<SegmentGroup [<Segment seg_0_Protein_chain_A>, <Segment seg_1_Protein_chain_A2>, <Segment seg_2_Protein_chain_A3>, <Segment seg_3_Protein_chain_A4>]>


In [5]:
# list of  Chain-IDs (A-Z)
chain_ids= [ 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 
             'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T',
             'U', 'V', 'W', 'X', 'Y', 'Z' ]

# loop over segments (chains)
for seg in u.segments:
    print("before:   segindex: {}  segid: {}".format(seg.segindex, seg.segid))
    
    # assign chainID to current segment
    seg.segid = chain_ids[ seg.segindex ]

    print("after:    segindex: {}  segid: {}\n".format(seg.segindex, seg.segid))


before:   segindex: 0  segid: seg_0_Protein_chain_A
after:    segindex: 0  segid: A

before:   segindex: 1  segid: seg_1_Protein_chain_A2
after:    segindex: 1  segid: B

before:   segindex: 2  segid: seg_2_Protein_chain_A3
after:    segindex: 2  segid: C

before:   segindex: 3  segid: seg_3_Protein_chain_A4
after:    segindex: 3  segid: D



In [6]:
# export fixed structure to PDB file
u.atoms.write("ldh_fixed_chainIDs.pdb")

/home/ostueker/.local/anaconda3/envs/mda/lib/python3.7/site-packages/MDAnalysis/coordinates/PDB.py:916: UserWarning: Found no information for attr: 'altLocs' Using default value of ' '
  "".format(attrname, default))
/home/ostueker/.local/anaconda3/envs/mda/lib/python3.7/site-packages/MDAnalysis/coordinates/PDB.py:916: UserWarning: Found no information for attr: 'icodes' Using default value of ' '
  "".format(attrname, default))
/home/ostueker/.local/anaconda3/envs/mda/lib/python3.7/site-packages/MDAnalysis/coordinates/PDB.py:916: UserWarning: Found no information for attr: 'occupancies' Using default value of '1.0'
  "".format(attrname, default))
/home/ostueker/.local/anaconda3/envs/mda/lib/python3.7/site-packages/MDAnalysis/coordinates/PDB.py:916: UserWarning: Found no information for attr: 'tempfactors' Using default value of '0.0'
  "".format(attrname, default))
